In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
# load train data
train_data = np.loadtxt("data/mnist_train.csv", delimiter=',', dtype=int)
test_data = np.loadtxt("data/mnist_test.csv", delimiter=',', dtype=int)
X_train, Y_train = train_data[:, 1:], train_data[:, 0]
X_test, Y_test = test_data[:, 1:], test_data[:, 0]
print(train_data.shape, test_data.shape)
print(X_train.shape, Y_train.shape)

In [ ]:
def draw_image_by_row(image_data):
    if image_data.shape[0] != 28 * 28:
        raise "Incorrect Image Size"

    image_2d = image_data.reshape(28, 28)
    # Отображаем изображение
    plt.imshow(image_2d, cmap='gray')  # используем 'gray' для монохромного изображения
    plt.axis('off')  # отключаем оси
    plt.show()


row_index = 0

print("Image of number:", Y_train[row_index])
draw_image_by_row(X_train[row_index])

In [ ]:
# Normalization
X_train = X_train/255
X_test = X_test/255

In [ ]:
def draw_learning_curves(train_losses, test_losses, train_accuracies, test_accuracies):
    # Построение графиков
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # График точности
    ax1.plot(range(epochs), train_accuracies, label="Train Accuracy")
    ax1.plot(range(epochs), test_accuracies, label="Test Accuracy")
    ax1.set_title("Accuracy per Epoch (Train vs Test)")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.legend()
    ax1.grid(True)

    # График потерь
    ax2.plot(range(epochs), train_losses, label="Train Loss")
    ax2.plot(range(epochs), test_losses, label="Test Loss")
    ax2.set_title("Loss per Epoch (Train vs Test)")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

**Классический подход:**
обучили, посмотрели результаты

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=[100],
    batch_size=64,
)
mlp.fit(X_train, Y_train)

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=[100],
    batch_size=64,
    max_iter=1,
    warm_start=True,
)

for i in range(20):
    mlp.fit(X_train, Y_train)

In [ ]:
y_pred = mlp.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
loss_curve = mlp.loss_curve_
train_accuracy = mlp.score(X_train, Y_train)
test_accuracy = mlp.score(X_test, Y_test)

fig, ax = plt.subplots(1, 2, figsize=(15, 6))

ax[0].plot(loss_curve, label="Loss")
ax[0].set_title("Learning Curve (Loss)")
ax[0].set_xlabel("Iterations")
ax[0].set_ylabel("Loss")
ax[0].legend()

ax[1].plot(np.arange(len(loss_curve)), [train_accuracy] * len(loss_curve), label="Train Accuracy", linestyle='--')
ax[1].plot(np.arange(len(loss_curve)), [test_accuracy] * len(loss_curve), label="Test Accuracy", linestyle='-.')
ax[1].set_title("Learning Curve (Accuracy)")
ax[1].set_xlabel("Iterations")
ax[1].set_ylabel("Accuracy")
ax[1].legend()

plt.tight_layout()
plt.show()

**Итеративно обучаем модель**, собираем метрики после каждой эпохи

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=[100],
    max_iter=1,
    warm_start=True,
    batch_size=64
)

train_accuracies = []
test_accuracies = []
train_losses = []
test_losses = []

epochs = 35
mlp.fit(X_train, Y_train)
for epoch in range(epochs):
    mlp.partial_fit(X_train, Y_train)

    train_accuracy = accuracy_score(Y_train, mlp.predict(X_train))
    test_accuracy = accuracy_score(Y_test, mlp.predict(X_test))

    train_loss = log_loss(Y_train, mlp.predict_proba(X_train))
    test_loss = log_loss(Y_test, mlp.predict_proba(X_test))

    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    train_losses.append(train_loss)
    test_losses.append(test_loss)

draw_learning_curves(train_losses, test_losses, train_accuracies, test_accuracies)
print(f"Test accuracy: {train_accuracies[-1] * 100:.2f}%")
print(f"Test  loss: {train_losses[-1]:.5f}")

In [ ]:
param_grid = {
    'hidden_layer_sizes': [[20], [100], [100, 20], [20, 100]],
    'activation': ['relu', 'tanh', 'logistic'],
    'batch_size': [64],
    'max_iter': [100],
}

mlp = MLPClassifier()
grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=3,
)
grid_search.fit(X_train, Y_train)
print("Best params:", grid_search.best_params_)
print("Final score (test data):", grid_search.score(X_test, Y_test))


In [ ]:
param_grid = {
    'hidden_layer_sizes': [[100, 20], [30, 80], [150, 40], [100, 50, 20]],
    'activation': ['relu', 'tanh', 'logistic'],
    'batch_size': [64],
    'max_iter': [100],
}

mlp = MLPClassifier()
grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=3,
)
grid_search.fit(X_train, Y_train)
print("Best params:", grid_search.best_params_)
print("Final score (test data):", grid_search.score(X_test, Y_test))
